In [ ]:
## environment preparation
##============================

%reset -f
%connect_info #might be used for "jupyter-console"

In [ ]:
#########################
## Load libraries
#########################

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import serial
import time
from tqdm import tqdm # progressbar in for loop
from collections import Counter # to sum dict values up
from datetime import datetime
import os

In [ ]:
## Define the classes to be measured
##=====================================
## teh values can be found in 04_TF_Train.ipynb" Cell 7

classes_values = ['beer',
                  'coffee',
                  'orange'
                  ]

# classes_values = ['sampleX1',
#                   'sampleX2',
#                   'sampleX3'
#                   ]

In [ ]:
#########################
## set the paths and load data
#########################
path_ipynb = os.getcwd()
path_base = path_ipynb + '/../../data/'

path_add_folder2readmodel = "tf_models/"
model_filename = 'model_demo_run'

In [ ]:
#########################
## Load model
#########################
model_ff = tf.keras.models.load_model((path_base + path_add_folder2readmodel + model_filename))

# Check its architecture
model_ff.summary()

In [ ]:
#########################
## Manual predict
#########################
#### The values are taken from "04_TF_Train.ipynb"

# Values_Train: [[64. 15. 64. 93. 15. 66.]]
# Label_Train:  [[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
# Label_Text:   [['cheese']]
data2predictArr = np.array([[64, 15, 64, 93, 15, 66]])

# Values_Train: [[ 82.  27.  70. 106.  22.  56.]]
# Label_Train:  [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
# Label_Text:   [['carrot']]
# data2predictArr = np.array([[ 82,  27,  70, 106,  22,  56]])
    
# Values_Test: [[ 60.  32. 154. 288.  25.  45.]]
# Label_Test:  [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
# Label_Text:   [['beer']]    
# data2predictArr = np.array([[ 60,  32, 154, 288,  25,  45]])    

# Values_Test: [[62. 33. 90. 91. 22. 60.]]
# Label_Test:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
# Label_Text:   [['sprite']]
# data2predictArr = np.array([[62, 33, 90, 91, 22, 60]])

predictionN = model_ff.predict(data2predictArr)
predictionN

In [ ]:
#########################
## Plot Manual predict
#########################
# fig = plt.figure(figsize=(6, 5))
# plt.barh(classes_values, predictionN.tolist()[0])
# for index, value in zip(classes_values, predictionN.tolist()[0]):
#     plt.text(0.7, index, str(round(value, 5)))
#     # plt.text(value, index, str(value))
# plt.tick_params(axis='x', colors='red')
# plt.tick_params(axis='y', colors='red')
# plt.grid()
# plt.show()

In [ ]:
#########################
## Live predict
#########################
N_of_readings = 50 #10Hz=100ms : 10 -> 1sec

serial_port = '/dev/ttyACM0'
baud_rate = 9600


pbar = tqdm(total=N_of_readings)
GGS_list = []
init_time = round(time.time(),3)*1000

with serial.Serial(serial_port, baud_rate) as ser:
    for cc1 in tqdm(range(10)):
        line_z = ser.readline();
        line_dec_z = line_z.decode("utf-8")
        
    while len(GGS_list) < N_of_readings:
        line = ser.readline();
        line_dec = line.decode("utf-8") #ser.readline returns a binary, convert to string
        lst0 = line_dec.split(",")
        lst1 = [int(x) for x in lst0]
        if len(lst1)==6:
            GGS_list += [[int(round(time.time(),3)*1000-init_time)] + lst1]
            pbar.update(1)
#             print(line_dec);

GGS_df = pd.DataFrame(GGS_list, columns=["time_ms", "B102NO2", "B302C2H5OH", "B502VOC", "B702CO", "TdegC", "RH"])
GGS_df

In [ ]:
##################################################
## Plot the collected data
##################################################
plt.plot(GGS_df.time_ms, GGS_df.iloc[:,1:7])
plt.legend(GGS_df.iloc[:,1:7].columns.to_list())
plt.tick_params(axis='x', colors='red')
plt.tick_params(axis='y', colors='red')
plt.grid()

In [ ]:
##################################################
## Predict the live data
##################################################
data2predictArr = GGS_df.iloc[:,1:].to_numpy()
data2predictArr.shape=(data2predictArr.shape[0],6)
predictionN = model_ff.predict(data2predictArr)
print(predictionN)
data2predictArrAver = predictionN.mean(axis=0)
print(data2predictArrAver)

In [ ]:
##################################################
## Plot the prediction of the live data
##################################################
fig = plt.figure(figsize=(6, 5))
plt.barh(classes_values, data2predictArrAver.tolist())
for index, value in zip(classes_values, predictionN.tolist()[0]):
    plt.text(0.7, index, str(round(value, 5)))
    # plt.text(value, index, str(value))
plt.tick_params(axis='x', colors='red')
plt.tick_params(axis='y', colors='red')
plt.grid()
plt.show()